### Agents with memory

In [2]:
from dataclasses import dataclass

from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

from dotenv import load_dotenv
import requests
import os

In [3]:
load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "langchain_agents_memory"
openai_model = init_chat_model(model="gpt-4.1-mini")

In [4]:
@dataclass
class Context:
    user_id: str

@dataclass
class ResponseFormat:
    summary: str
    temperature_celsius: float
    temperature_fahrenheit: float
    humidity: float

In [10]:
@tool
def get_weather(city: str) -> str:
    """Returns the weather of a given city"""
    response = requests.get(f"https://wttr.in/{city}?format=j1")
    return response.json()

@tool
def locate_user(runtime: ToolRuntime[Context]) -> str:
    """Returns the location of a user based on context"""
    match runtime.input.user_id:
        case "11":
            return "New York"
        case "12":
            return "London"
        case "13":
            return "San Francisco"
        case _:
            return "Dallas"

In [11]:
checkpointer = InMemorySaver()

In [12]:
agent = create_agent(
    model=openai_model, 
    tools=[get_weather, locate_user], 
    system_prompt="You are helpful weather agent, give consize description of the weather",
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

In [14]:
# now when we call the agent we need to pass the context

config = {"configurable": {"thread_id": "123"}}

response = agent.invoke({
    "messages": [
        {"role": "user", "content": "what is the weather like?"}
    ]},
    context=Context(user_id="13"),
    config=config
)

print(response["structured_response"])

ResponseFormat(summary='The weather is sunny and clear with a temperature around 7°C (44°F), low humidity at 40%, and moderate westward winds at 27 km/h (17 mph).', temperature_celsius=7.0, temperature_fahrenheit=44.0, humidity=40.0)
